In [1]:
import openseespy.opensees as ops
import numpy as np
import scipy.linalg as slin

ops.database('File','C:\\Users\\AETL\\Documents\\GitHub\\OpenSeesScript\\Ohtori20Story\\model\\tcl\\save\\20story')
ops.restore(1)

FATAL - FileDatastore::openFile() - could not open file C:\Users\AETL\Documents\GitHub\OpenSeesScript\Ohtori20Story\model\save\20story.IDs.15.1
FileDatastore::recvID() - could not open file
Domain::recv - channel failed to recv the initial ID
FE_Datastore::restoreState - domain failed to recvSelf
FATAL - FileDatastore::openFile() - could not open file C:\Users\AETL\Documents\GitHub\OpenSeesScript\Ohtori20Story\model\save\20story.IDs.1.0
FileDatastore::recvID() - could not open file
FE_Datastore::restoreState - failed to get max lastDbTag data from database - problems may ariise
WARNING - database failed to restore state 


OpenSeesError: See stderr output

In [ ]:
Nmodes = 6 
w2 = ops.eigen(Nmodes)

# Pick your modes and damping ratios
wi = w2[0]**0.5; zetai = 0.02 # 5% in mode 1
wj = w2[5]**0.5; zetaj = 0.02 # 2% in mode 3

import numpy as np

A = np.array([[1/wi, wi],[1/wj, wj]])
b = np.array([zetai,zetaj])

x = np.linalg.solve(A,2*b)

#             M    KT  KI  Kn
ops.rayleigh(x[0],0.0,0.0,x[1])

WARNING PlainHandler::handle() -  constraint at dof 0 already specified for constrained node in MP_Constraint at node 2011
WARNING PlainHandler::handle() -  constraint at dof 0 already specified for constrained node in MP_Constraint at node 3011
WARNING PlainHandler::handle() -  constraint at dof 0 already specified for constrained node in MP_Constraint at node 4011
WARNING PlainHandler::handle() -  constraint at dof 0 already specified for constrained node in MP_Constraint at node 5011
WARNING PlainHandler::handle() -  constraint at dof 0 already specified for constrained node in MP_Constraint at node 6011


In [2]:
ops.wipeAnalysis()
ops.system('FullGeneral')
ops.analysis('Transient')
ops.numberer('Plain')
# Mass
ops.integrator('GimmeMCK',1.0,0.0,0.0)
ops.analyze(1,0.0)
 
# Number of equations in the model
N = ops.systemSize() # Has to be done after analyze
 
M = ops.printA('-ret') # Or use ops.printA('-file','M.out')
M = np.array(M) # Convert the list to an array
M.shape = (N,N) # Make the array an NxN matrix
 
# Stiffness
ops.wipeAnalysis()
ops.system('FullGeneral')
ops.analysis('Transient')
ops.numberer('Plain')


ops.integrator('GimmeMCK',0.0,0.0,1.0)
ops.analyze(1,0.0)
K = ops.printA('-ret')
K = np.array(K)
K.shape = (N,N)
 
# Damping
ops.wipeAnalysis()
ops.system('FullGeneral')
ops.analysis('Transient')
ops.numberer('Plain')

ops.integrator('GimmeMCK',0.0,1.0,0.0)
ops.analyze(1,0.0)
C = ops.printA('-ret')
C = np.array(C)
C.shape = (N,N)

: 

: 

In [4]:
np.savetxt('fullM.txt',M)
np.savetxt('fullC.txt',C)
np.savetxt('fullK.txt',K)

In [24]:
for nd in ops.getNodeTags():
     ops.nodeDOFs(nd,1)

In [25]:
XDofs = []
for nd in ops.getNodeTags():
     if (ops.nodeDOFs(nd,1)[0] >= 0):
          XDofs.append(ops.nodeDOFs(nd,1)[0])
     

In [26]:
XDofs

[1,
 6,
 9,
 12,
 15,
 18,
 21,
 24,
 27,
 30,
 33,
 36,
 39,
 42,
 45,
 48,
 51,
 54,
 57,
 60,
 63,
 67,
 6,
 9,
 12,
 15,
 18,
 21,
 24,
 27,
 30,
 33,
 36,
 39,
 42,
 45,
 48,
 51,
 54,
 57,
 60,
 63,
 113,
 6,
 9,
 12,
 15,
 18,
 21,
 24,
 27,
 30,
 33,
 36,
 39,
 42,
 45,
 48,
 51,
 54,
 57,
 60,
 63,
 159,
 6,
 9,
 12,
 15,
 18,
 21,
 24,
 27,
 30,
 33,
 36,
 39,
 42,
 45,
 48,
 51,
 54,
 57,
 60,
 63,
 205,
 6,
 9,
 12,
 15,
 18,
 21,
 24,
 27,
 30,
 33,
 36,
 39,
 42,
 45,
 48,
 51,
 54,
 57,
 60,
 63,
 251,
 6,
 9,
 12,
 15,
 18,
 21,
 24,
 27,
 30,
 33,
 36,
 39,
 42,
 45,
 48,
 51,
 54,
 57,
 60,
 63,
 1,
 67,
 1,
 113,
 1,
 159,
 1,
 205,
 1,
 1]

In [7]:
myset = set(XDofs)
XDofs = list(myset)

In [8]:
for nd in ops.getNodeTags():
    print(nd, ops.nodeDOFs(nd))

100 [-1, -1, 0]
101 [1, 2, 3]
102 [-1, 4, 5]
103 [6, 7, 8]
104 [9, 10, 11]
105 [12, 13, 14]
106 [15, 16, 17]
107 [18, 19, 20]
108 [21, 22, 23]
109 [24, 25, 26]
110 [27, 28, 29]
111 [30, 31, 32]
112 [33, 34, 35]
113 [36, 37, 38]
114 [39, 40, 41]
115 [42, 43, 44]
116 [45, 46, 47]
117 [48, 49, 50]
118 [51, 52, 53]
119 [54, 55, 56]
120 [57, 58, 59]
121 [60, 61, 62]
122 [63, 64, 65]
200 [-1, -1, 66]
201 [67, 68, 69]
202 [-1, 70, 71]
203 [6, 72, 73]
204 [9, 74, 75]
205 [12, 76, 77]
206 [15, 78, 79]
207 [18, 80, 81]
208 [21, 82, 83]
209 [24, 84, 85]
210 [27, 86, 87]
211 [30, 88, 89]
212 [33, 90, 91]
213 [36, 92, 93]
214 [39, 94, 95]
215 [42, 96, 97]
216 [45, 98, 99]
217 [48, 100, 101]
218 [51, 102, 103]
219 [54, 104, 105]
220 [57, 106, 107]
221 [60, 108, 109]
222 [63, 110, 111]
300 [-1, -1, 112]
301 [113, 114, 115]
302 [-1, 116, 117]
303 [6, 118, 119]
304 [9, 120, 121]
305 [12, 122, 123]
306 [15, 124, 125]
307 [18, 126, 127]
308 [21, 128, 129]
309 [24, 130, 131]
310 [27, 132, 133]
311 [30, 13

In [29]:
len(masslessDOFs)

10

In [27]:
# Determine number of DOFs with mass
 
# Number of DOFs with mass
massDOFs = np.where(np.diag(M) != 0)[0]
#massDOFs = XDofs
#massDOFs = [6,9,12,15,18,21,24,27,30,33,36,39,42,45,48,51,54,57,60,63,]
Nmass = len(massDOFs)
 
# DOFs without mass
masslessDOFs = np.where(np.diag(M) == 0)[0]
#d = [x for x in range(N)]  
#masslessDOFs = [x for x in d if x not in massDOFs]
Nmassless = len(masslessDOFs)
# Form matrices for D*x = -lam*B*x
B = np.zeros((2*Nmass,2*Nmass)) # = [ 0 M; M C]
D = np.zeros((2*Nmass,2*Nmass)) # = [-M 0; 0 K]
 
# Mass
B[:Nmass,:][:,Nmass:2*Nmass] =  M[massDOFs,:][:,massDOFs]
B[Nmass:2*Nmass,:][:,:Nmass] =  M[massDOFs,:][:,massDOFs]
D[:Nmass,:][:,:Nmass]        = -M[massDOFs,:][:,massDOFs]
 
# Damping
B[Nmass:2*Nmass,:][:,Nmass:2*Nmass] = C[massDOFs,:][:,massDOFs]
 
# Static condensation
Kmm = K[massDOFs,:][:,massDOFs];     Kmn = K[massDOFs,:][:,masslessDOFs]
Knm = K[masslessDOFs,:][:,massDOFs]; Knn = K[masslessDOFs,:][:,masslessDOFs]
# Kc = Kmm - Kmn*inv(Knn)*Knm
if Nmassless > 0:
    Kc = Kmm - np.dot(Kmn,np.linalg.solve(Knn,Knm))
else:
    Kc = K

np.savetxt('npM.txt',M[massDOFs,:][:,massDOFs])
np.savetxt('npC.txt',C[massDOFs,:][:,massDOFs])
np.savetxt('npK.txt',Kc)

# Stiffness at DOFs with mass
D[Nmass:2*Nmass,:][:,Nmass:2*Nmass] = Kc
 
# State space eigenvalue analysis
x = slin.eigvals(D,-B)

In [19]:
len(massDOFs)

296

In [7]:
np.savetxt('npM.txt',M[massDOFs,:][:,massDOFs])
np.savetxt('npC.txt',C[massDOFs,:][:,massDOFs])
np.savetxt('npK.txt',Kc)